# Import

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score


# Reduce File Size into chunks

In [ ]:
"""
# Load the large CSV file
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Data/training_set_VU_DM.csv'
data = pd.read_csv(file_path)

# Define the size of each chunk
chunk_size = 500000  # This number can change 

# Split the CSV into chunks
for i in range(0, len(data), chunk_size):
    chunk = data.iloc[i:i + chunk_size]
    chunk.to_csv(f'chunk_{i//chunk_size}.csv', index=False)

"""

In [ ]:
df = pd.read_csv("chunk_0.csv")
df['date_time'] = pd.to_datetime(df['date_time'])
df.head()

In [ ]:
print(len(df.columns))

df.columns

# Data cleaning

要增加的部分

1，如果一个人只浏览但没有点击，那么它的数据在排序中是否是无意义的？

## Remove missing values

In [27]:
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/chunk_0_cleaned.csv'

df_cleaned_chunk0 = pd.read_csv(file_path)

In [28]:
print(df_cleaned_chunk0.head())

   srch_id            date_time  site_id  visitor_location_country_id  \
0        1  2013-04-04 08:32:15       12                          187   
1        1  2013-04-04 08:32:15       12                          187   
2        1  2013-04-04 08:32:15       12                          187   
3        1  2013-04-04 08:32:15       12                          187   
4        1  2013-04-04 08:32:15       12                          187   

   visitor_hist_starrating  visitor_hist_adr_usd  prop_country_id  prop_id  \
0                      0.0                   0.0              219      893   
1                      0.0                   0.0              219    10404   
2                      0.0                   0.0              219    21315   
3                      0.0                   0.0              219    27348   
4                      0.0                   0.0              219    29604   

   prop_starrating  prop_review_score  ...  comp7_rate  comp7_inv  \
0                3     

## Classify according to different variable types

In [ ]:
Criteria_in_the_search_query = ['date_time', 'site_id','srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool']

Static_hotel_characteristics = ['prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price','srch_query_affinity_score','orig_destination_distance']

Dynamic_hotel_characteristics = ['random_bool', 'position', 'price_usd', 'promotion_flag','click_bool', 'gross_bookings_usd',
       'booking_bool']

Visitor_information = ['srch_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd']

Competitive_information = ['comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff']

Other = []

all_catogories = [Criteria_in_the_search_query, Static_hotel_characteristics, 
                 Dynamic_hotel_characteristics, Visitor_information,Competitive_information,Other]
all_variables = []
for i in all_catogories:
    for j in i:
        all_variables.append(j)

df_variables = df.columns.to_list()

differences = set(all_variables).symmetric_difference(set(df_variables))

if differences == set():
    print('All catogorized')
else:
    print(f'Differences between df_variables and catogorized variables: {differences}')


### Criteria_in_the_search_query

In [ ]:
print(Criteria_in_the_search_query)

# Model lambdarank

## Basic model(without parameter choose and feature engineering)

In [ ]:
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/chunk_0_cleaned.csv'

#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/cleaned_training_set_VU_DM.csv'
#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/training_set_VU_DM.csv'
df = pd.read_csv(file_path)

# 增加一个计算得分的列
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

df['score'] = df.apply(assign_scores, axis=1)

features = ['site_id','srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool','prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price',

       'srch_query_affinity_score','orig_destination_distance','random_bool', 
       'price_usd', 'promotion_flag','srch_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd','comp1_rate', 

       'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate', 'comp8_inv',
       'comp8_rate_percent_diff']

# 划分数据集
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['srch_id'])

# 准备 LightGBM 数据结构
train_data = lgb.Dataset(train_df[features], label=train_df['score'], group=train_df['srch_id'].value_counts().sort_index())
test_data = lgb.Dataset(test_df[features], label=test_df['score'], group=test_df['srch_id'].value_counts().sort_index())

# 设置模型参数
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [3, 5],
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1
}

# 训练模型
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# 预测测试集
test_pred = bst.predict(test_df[features])

# 评估模型，计算 NDCG 分数
test_df['predictions'] = test_pred

# 首先确保数据按照 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []

for name, group in grouped:
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")


### Result output

In [ ]:
# 使用迭代器逐块读取数据
chunk_size = 10000
reader = pd.read_csv('C:/Users/98398/Desktop/P5/DM-AS2/Data/test_set_VU_DM.csv', chunksize=chunk_size)

predictions = []  # 创建一个空列表以存储每个块的预测结果
for chunk in reader:
    # 可以在这里添加数据预处理步骤，比如填充缺失值等
    chunk_pred = bst.predict(chunk[features])  # 应用模型进行预测
    chunk['predictions'] = chunk_pred  # 将预测结果添加到 DataFrame
    predictions.append(chunk[['srch_id', 'prop_id', 'predictions']])  # 仅保留需要的列

# 合并所有批次的预测结果
final_predictions = pd.concat(predictions)

In [ ]:
# 确保按照预测分数排序，如果 Kaggle 要求
final_predictions.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 选择需要的列
final_predictions = final_predictions[['srch_id', 'prop_id']]

# 保存为 CSV 文件，确保不包含索引，包含列标题
final_predictions.to_csv('train=all_cleaned.csv', index=False, header=True)

In [ ]:
print(final_predictions)

### All train data

In [4]:
All_test_df = pd.read_csv('C:/Users/98398/Desktop/P5/DM-AS2/Data/training_set_VU_DM.csv')

# 预测测试集

All_test_pred = bst.predict(All_test_df[features])

# 评估模型，计算 NDCG 分数
All_test_df['predictions'] = All_test_pred

All_test_df['score'] = All_test_df.apply(assign_scores, axis=1)
# 假设 df 是你的 DataFrame
# 首先确保数据按照 srch_id 和 predictions 降序排序
All_test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = All_test_df.groupby('srch_id')
ndcg_scores = []

for name, group in grouped:
    # 确保每组数据中的预测是降序的（虽然已全局排序，这里是双重确认）
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

# 计算平均 NDCG 分数
average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")


Average NDCG Score: 0.3288798428994425


## Use all cleaned data without feature engineering

## With feature engineering by Menghan 

### Model build

In [18]:
file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/feature_engineered_training_chunk_0.csv'

#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/cleaned_training_set_VU_DM.csv'
#file_path = 'C:/Users/98398/Desktop/P5/DM-AS2/Test_data/training_set_VU_DM.csv'
df = pd.read_csv(file_path)

# 增加一个计算得分的列
def assign_scores(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

df['score'] = df.apply(assign_scores, axis=1)


281


In [33]:
columns = df.columns

# 使用列表推导式筛选出不需要删除的列
features = [
    col for col in columns if col not in ['date_time', 'position', 'click_bool', 'booking_bool', 'score']
    and 'gross_bookings_usd' not in col and 'position' not in col
]


print(len(features))
print(len(df.columns))

257
281


In [34]:
# 划分数据集
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['srch_id'])

# 准备 LightGBM 数据结构
train_data = lgb.Dataset(train_df[features], label=train_df['score'], group=train_df['srch_id'].value_counts().sort_index())
test_data = lgb.Dataset(test_df[features], label=test_df['score'], group=test_df['srch_id'].value_counts().sort_index())

# 设置模型参数
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'ndcg_eval_at': [3, 5],
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1
}

# 训练模型
num_round = 100
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# 预测测试集
test_pred = bst.predict(test_df[features])

# 评估模型，计算 NDCG 分数
test_df['predictions'] = test_pred

# 首先确保数据按照 srch_id 和 predictions 降序排序
test_df.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 分组并计算每个搜索会话的 NDCG
grouped = test_df.groupby('srch_id')
ndcg_scores = []

for name, group in grouped:
    group = group.sort_values('predictions', ascending=False)
    true_relevance = group['score'].values
    scores_pred = group['predictions'].values
    # 计算当前搜索会话的 NDCG 分数，并追加到列表中
    if len(np.unique(true_relevance)) > 1:  # 只计算有正样本的会话
        ndcg_scores.append(ndcg_score([true_relevance], [scores_pred], k=5))

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG Score: {average_ndcg}")


Average NDCG Score: 0.7303758851941361


### Result output

In [ ]:
# 使用迭代器逐块读取数据
chunk_size = 10000
reader = pd.read_csv('C:/Users/98398/Desktop/P5/DM-AS2/Data/test_set_VU_DM.csv', chunksize=chunk_size)

predictions = []  # 创建一个空列表以存储每个块的预测结果
for chunk in reader:
    # 可以在这里添加数据预处理步骤，比如填充缺失值等
    chunk_pred = bst.predict(chunk[features])  # 应用模型进行预测
    chunk['predictions'] = chunk_pred  # 将预测结果添加到 DataFrame
    predictions.append(chunk[['srch_id', 'prop_id', 'predictions']])  # 仅保留需要的列

# 合并所有批次的预测结果
final_predictions = pd.concat(predictions)

In [ ]:
# 确保按照预测分数排序，如果 Kaggle 要求
final_predictions.sort_values(['srch_id', 'predictions'], ascending=[True, False], inplace=True)

# 选择需要的列
final_predictions = final_predictions[['srch_id', 'prop_id']]

# 保存为 CSV 文件，确保不包含索引，包含列标题
final_predictions.to_csv('train=all_cleaned.csv', index=False, header=True)

In [ ]:
print(final_predictions)